In [ ]:
#Author: Zhengyi Cui
#Created time: 2024-11-13
#Email: Zhengyi.Cui@uth.tmc.edu

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.img_tiles as cimgt
import geopandas
import matplotlib.patches as mpatches
import matplotlib.patheffects as pe
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from matplotlib.colors import ListedColormap, BoundaryNorm


data = pd.read_csv('./epa_pm25.csv')
county_df = pd.read_csv(
    './co-est2023-alldata.csv',
    encoding='latin1',
)

fig, ax = plt.subplots(figsize=(12, 8), subplot_kw={'projection': ccrs.PlateCarree()})
county_df=county_df[county_df['STNAME']=='Texas']
county_df = county_df[['STATE', 'COUNTY', 'POPESTIMATE2023']]
county_df['COUNTY'] = county_df['COUNTY'].apply(lambda x: str(x).zfill(3))
counties = geopandas.read_file("https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json")
texas_counties = counties[counties['STATE'] == '48'] 
texas_counties = texas_counties.merge(county_df,how='inner',on='COUNTY')

ax.set_extent([-107, -93, 25, 37], crs=ccrs.PlateCarree())
osm = cimgt.OSM()

min_pop = texas_counties['POPESTIMATE2023'].min()
max_pop = texas_counties['POPESTIMATE2023'].max()

bounds = [0, 10000, 50000, 100000, 200000, 500000, 1000000, max_pop]

colors = [
    (255/255, 255/255, 190/255),  # light yellow
    (255/255, 255/255, 115/255),  # medium yellow
    (255/255, 255/255, 0/255),    # yellow
    (255/255, 170/255, 0/255),    # orange
    (230/255, 76/255, 0/255),     # vermillion
    (230/255, 0/255, 0/255),      # red
    (115/255, 0/255, 0/255)       # maroon
]

legend_labels = [
    '0 - 10,000',
    '10,001 - 50,000',
    '50,001 - 100,000',
    '100,001 - 200,000',
    '200,001 - 500,000',
    '500,001 - 1,000,000',
    f'1,000,001 - {int(max_pop):,}'
]

cmap = ListedColormap(colors)
norm = BoundaryNorm(bounds, ncolors=len(colors))

osm = cimgt.OSM()
ax.add_image(osm, 8)


texas_counties.plot(
    column='POPESTIMATE2023',
    ax=ax,
    cmap=cmap,
    norm=norm,
    alpha=0.6,
    edgecolor='black',
    linewidth=0.5
)


handles = [mpatches.Patch(facecolor=colors[i], edgecolor='black', label=legend_labels[i]) for i in range(len(legend_labels))]

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES.with_scale('10m'), linestyle='--')
texas_counties.boundary.plot(ax=ax, edgecolor='gray', linestyle='-', linewidth=0.5)

scatter = ax.scatter(data['longitude'], data['latitude'], cmap='viridis', alpha=0.7, edgecolor='k', s=40)
legend1 = ax.legend([scatter], ['AQS PM2.5 Ground Sites'], loc='upper right')
ax.add_artist(legend1)
ax.legend(handles=handles, title='population size', loc='lower right')
ax.text(-106, 36, 'N', ha='center', va='center', fontsize=16, fontweight='bold', transform=ccrs.PlateCarree())

# ax.text(-106.8, 25.7, 'https://www.census.gov', transform=ccrs.PlateCarree(),
#         fontsize=12, ha='left', va='bottom', color='blue')

arrow = mpatches.FancyArrowPatch(
    (-106, 35.5),  
    (-106, 35.9), 
    transform=ccrs.PlateCarree(), 
    color='black',
    arrowstyle='-|>',  
    linewidth=2,
    mutation_scale=15,  
    path_effects=[pe.withStroke(linewidth=2, foreground='white')]
)
ax.add_patch(arrow)

fontprops = fm.FontProperties(size=12)


scalebar = AnchoredSizeBar(
    ax.transData,
    1,                    
    '100 km',             
    pad=0.5,
    color='black',      
    frameon=False,
    size_vertical=0.02,    
    borderpad=0.5,        
    sep=5,                 
    fontproperties=fontprops
)

ax.add_artist(scalebar)

plt.title('Texas Population and AQS PM2.5 Monitoring Sites', fontsize=16)

plt.tight_layout()
plt.savefig('./Texas Population and AQS PM2.5 Monitoring Sites.png', dpi=300)
plt.show()